In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/output_example.c", 
    lang="", 
    number=True,
    title="<b>output_example.c - Uncontrolled thread interleaving using output to the terminal.</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>output_example.c - Uncontrolled thread interleaving using output to the terminal.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: 
7: /* This function is intended so simulate a thread needing to do some other 
8:  * work in between output statements.
9:  */
10: static long waste_cpu()
11: {
12:     long dummy = 0;
13:     for (int i = 0; i < 10000; i++) {
14:         dummy += rand();
15:     }
16:     return dummy;
17: }
18: 
19: /* Start of function executed by each thread.
20:  * Each character of the argument string is printed out one-by-one
21:  * in a separate system call. 
22:  * To increase the number of interleavings, we voluntarily yield the 
23:  * CPU after each character is output.
24:  */
25: static void *write_one_by_one(void *arg)
26: {
27:     char *mystr = (char *)arg;
28:       
29:     for (int i = 0; i < strlen(mystr); i++) {
30:         write(STDOUT_FILENO, &mystr[i], 1); 
31:         waste_cpu();
32:     }
33:   
34:     return (void *)0;       
35: }
36: /* End of function executed by each thread. */
37: 
38: 
39: int main(int argc, char **argv)
40: {
41:     pthread_t tid1, tid2;
42:     char *str1 = "Hello";
43:     char *str2 = "Goodbye";
44: 
45:     /* Normally we'd want to test the return value of pthread_create. */
46: 
47:     (void)pthread_create(&tid1, NULL, write_one_by_one, (void *)str1 );
48:     (void)pthread_create(&tid2, NULL, write_one_by_one, (void *)str2 );
49: 
50:     /* Wait for child threads to finish */
51:     pthread_join(tid1, NULL);
52:     pthread_join(tid2, NULL);
53: 
54:     (void)write(STDOUT_FILENO, "\n", 1);    
55:     return 0;
56: }
57: 

```


</div>
</font>

In [4]:
bash.runNoOutput("[[ -a output_example ]] && rm output_example;make output_example")
cmds = '''./output_example
./output_example
./output_example
./output_example'''

bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [5]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/bank_deposit.c", 
    lang="", 
    number=True,
    title="<b>bank_deposit.c - Full source code for simple bank account example</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>bank_deposit.c - Full source code for simple bank account example</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: 
7: #define money_t double
8: #define MAX_ACCOUNTS 1000
9: 
10: struct deposit_s {
11:     int account_num;
12:     money_t amount;
13: };
14: 
15: money_t accounts[MAX_ACCOUNTS];
16: 
17: /* This function is intended so simulate the work of retrieving the current balance
18:  * for a given account number. It just spins and yields a random number of times.
19:  */
20: static money_t get_balance(int account_num)
21: {
22:     long dummy = 0;
23:     int nyield = rand() % 10;
24:     int nspin = rand() % 1000;
25: 
26: 
27:     for (int i = 0; i < nyield; i++) {
28:         sched_yield();
29:         for (int j = 0; j < nspin; j++) {
30:             dummy += rand();
31:         }
32:     }
33: 
34:     return accounts[account_num];
35: }
36: 
37: /* This function is intended so simulate the work of saving the current balance
38:  * back to a given account number. It just spins and yields a random number of times.
39:  */
40: static void put_balance(int account_num, money_t balance)
41: {
42:     long dummy = 0;
43:     int nyield = rand() % 10;
44:     int nspin = rand() % 1000;
45: 
46: 
47:     for (int i = 0; i < nyield; i++) {
48:         sched_yield();
49:         for (int j = 0; j < nspin; j++) {
50:             dummy += rand();
51:         }
52:     }
53: 
54:     accounts[account_num] = balance;
55:     return;
56: }
57: 
58: /* Start of deposit function */
59: static void deposit(int account, money_t amount)
60: {
61:     money_t balance = get_balance(account);
62:     balance += amount;
63:     put_balance(account, balance);
64: 
65:     return;        
66: }
67: /* End of deposit function */
68: 
69: static void *deposit_thread(void *arg)
70: {
71:     struct deposit_s *my_deposit = (struct deposit_s *)arg;
72:     deposit(my_deposit->account_num, my_deposit->amount);
73: 
74:     return (void *)0;
75: }
76: 
77: int main(int argc, char **argv)
78: {
79:     pthread_t tid1, tid2;
80:     int account_num = 100;
81:     money_t amt1 = 50.00;
82:     money_t amt2 = 100.00; 
83: 
84:     struct deposit_s dep1 = {account_num, amt1};
85:     struct deposit_s dep2 = {account_num, amt2}; 
86: 
87:     /* Normally we'd want to test the return value of pthread_create. */
88: 
89:     (void)pthread_create(&tid1, NULL, deposit_thread, (void *)&dep1 );
90:     (void)pthread_create(&tid2, NULL, deposit_thread, (void *)&dep2 );
91: 
92:     /* Wait for child threads to finish */
93:     pthread_join(tid1, NULL);
94:     pthread_join(tid2, NULL);
95: 
96:     printf("Final balance of account #%d after depositing $%.2lf and $%.2lf is $%.2lf\n",
97:            account_num, amt1, amt2, accounts[account_num] );
98:     return 0;
99: }

```


</div>
</font>

In [6]:
bash.runNoOutput("[[ -a bank_deposit ]] && rm bank_deposit;make bank_deposit")
cmds = '''./bank_deposit
./bank_deposit
./bank_deposit
./bank_deposit
./bank_deposit'''
bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [7]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/counter.c", 
    lang="", 
    number=True,
    title="<b>counter.c - Two threads increment and decrement a shared counter without synchronization.</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>counter.c - Two threads increment and decrement a shared counter without synchronization.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: 
7: volatile long shared_counter;
8: 
9: static void *increment_thread(void *arg)
10: {
11:     long niters = (long)arg;
12:     for (long i = 0; i < niters; i++) {
13:         shared_counter++;
14:     }
15: 
16:     return (void *)0;
17: }
18: 
19: static void *decrement_thread(void *arg)
20: {
21:     long niters = (long)arg;
22: 
23:     for (long i = 0; i < niters; i++) {
24:         shared_counter--;
25:     }
26: 
27:     return (void *)0;
28: }
29: 
30: int main(int argc, char **argv)
31: {
32:     pthread_t *tids;
33:     long niters;
34:     int nthreads;
35:     
36:     if (argc != 3) {
37:         fprintf(stderr,"Usage: %s <num_threads> <num_iters>\n",argv[0]);
38:         return 1;
39:     }
40: 
41:     nthreads = atoi(argv[1]);
42:     niters = atoi(argv[2]);
43:     shared_counter = 0;
44:     
45:     printf("Main thread: Beginning test with %d threads\n", nthreads);
46:     
47:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
48:     
49:     /* We create the same number of increment and decrement threads, each doing the same number of iterations. 
50:      * When all threads have completed, we expect the final value of the shared counter to be the same as its
51:      * initial value (i.e., 0).
52:      */
53:     for (int i = 0; i < nthreads; i+=2) {
54:         (void)pthread_create(&tids[i], NULL, increment_thread, (void *)niters );
55:         (void)pthread_create(&tids[i+1], NULL, decrement_thread, (void *)niters );
56:     }
57:     
58:     /* Wait for child threads to finish */
59:     for (int i = 0; i < nthreads; i+=2) {
60:         pthread_join(tids[i], NULL);
61:         pthread_join(tids[i+1], NULL);
62:     }
63:     
64:     printf("Main thread: Final value of shared counter is %ld\n", shared_counter);
65: 
66:     return 0;
67: }

```


</div>
</font>

In [8]:
bash.runNoOutput("[[ -a counter ]] && rm counter;make counter")
cmds = '''./counter 2 20000
./counter 2 100000
./counter 4 100000
./counter 4 100000'''

bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…